In [ ]:


class MultiHeadAttention:
    def __init__(self, d,h):
        self.d = d
        self.h = h
        self.d_k = d/ h
        self.v_weights, self.k_weights, self.q_weights, self.lin_trans = (np.random.randn(4,d,d))
        self.bias_weights = np.zeros(d)
    

    def split_heads(self, x, batch_size):
        x = x.reshape(batch_size,-1,self.h,self.d_k)
        return x.transpose(0,2,1,3)
    def scaled_self_attention(self,queries,keys,values):
        attention_scores = np.matmul(queries, keys.transpose(0,2,1)) / np.sqrt(self.d_k)
        attention_weights = softmax(attention_scores)
        attention_outputs = np.matmul(attention_weights, values)
        return attention_output
        
    def forward(self, x):
        self.x = x  
        batch_size, seq_len,d = x.shape

        values = np.dot(x, self.v_weights)
        keys = np.dot(x, self.k_weights)
        queries = np.dot(x, self.q_weights)

        values = self.split_heads(values, batch_size)
        keys = self.split_heads(keys, batch_size)
        queries = self.split_heads(queries, batch_size)

        attention_output=self.scaled_self_attention(self,queries,keys,values)
        attention_output = attention_outputs.reshape(batch_size,-1)
        normalized_output = self.layer_norm(x + attention_output)
        ffn_output = np.dot(normalized_output, self.lin_trans) + self.bias_weights1
        ffn_output = np.maximum(0, ffn_output) 
        output = np.dot(ffn_output, self.lin_trans.T) + self.bias_weights2

        return output
    def backward(self, d_output): 
        d_ffn_output = np.dot(d_output, self.lin_trans)
        d_ffn_output[d_ffn_output <= 0] = 0
        d_normalized_output = np.dot(d_ffn_output, self.lin_trans)
        d_input = self.layer_norm_backward(d_normalized_output)
        d_gamma = np.sum(d_normalized_output * (self.x - self.layer_mean), axis=-1, keepdims=True)
        d_beta = np.sum(d_normalized_output, axis=-1, keepdims=True)
        return d_input, d_gamma, d_beta
